In [1]:
import numpy as np
import librosa, os, keras, time
import matplotlib.pyplot as plt
from pydub import AudioSegment
from pydub.utils import make_chunks
from collections import deque

Using TensorFlow backend.


In [69]:
dir1 = './Instrument-Classification/IRMAS-TestingData-Part1/Part1/'
dir2 = './Instrument-Classification/IRMAS-TestingData-Part2/Part2/'
dir3 = './Instrument-Classification/IRMAS-TestingData-Part3/Part3/'

model = keras.models.load_model('ICmodel_pggbvv.h5')
#model = keras.models.load_model('ICmodel_PGm_1024dense_adam.h5')

dir = dir1
instr = ['pia','gac','gel','buz','vio','voi']
#instr = ['pia,','gac']

In [66]:
def to_categorical(list_final):
    temp_list = [[0]*len(instr)]*(len(list_final))
    for i in range(len(list_final)):
        x = [0]*len(instr)
        x.insert(0,1)
        x.pop()
        for ins in instr:
            if (ins in list_final[i]): 
                temp_list[i] = [sum(x) for x in zip(temp_list[i],x)]
            x.insert(0,x.pop())
    return temp_list[0]

def prediction(predict):
    prediction = model.predict(predict)
    sum = np.zeros(len(prediction[0]))
    for pred in prediction:
        for i in range(len(pred)):
            sum[i] = sum[i] + pred[i]
    sum[:] = [round(x/len(prediction),2) for x in sum]
    return sum

def average_accuracy(y_pred,y_label):
    acc = 0
    for i in range(len(y_pred)):
        acc = acc + abs(y_pred[i]-y_label[i])
    acc = acc / len(y_pred)
    return round(100*(1-acc),2)

def existance_accuracy(y_pred,y_label):
    acc = 0
    for i in range(len(y_pred)):
        if (y_pred[i]>0.5 and y_label[i]==1): acc+=1
        elif (y_pred[i]>0.5 and y_label[i]==0): acc-=1
        elif (y_pred[i]<0.5 and y_label[i]==0): acc+=1
        elif (y_pred[i]<0.5 and y_label[i]==1): acc-=1
    acc = acc / len(y_pred)
    return round(100*(acc),2)
    
def predict_songs(start,end):
    sum_acc1 = 0
    sum_acc2 = 0
    it = 1
    start = 2*start
    end = 2*end - start
    text = True
    unk_instr = False
    for f in os.listdir(dir):
    
        if (start!=0):
            start = start -1
            continue
    
        if (unk_instr):
            unk_instr = False
            continue
        
        if (text):
            y_test = []
            x = open(dir+f)
            for line in x.readlines():
                y_test.append(line[:3])
                if (not (line[:3] in instr)): 
                    unk_instr = True
                    print("Unknown instrument in: ",f)
                    print("")
            if (unk_instr): 
                continue
            y_label = to_categorical([y_test])
        
        if (not text):
            predict = []
            wav,sr = librosa.load(dir+f,mono=True,sr=22050)
            wav = wav / np.sqrt(np.mean(wav**2))
            chunks = make_chunks(wav,sr)
            for ch in chunks[:-1]:
                    mel = librosa.feature.melspectrogram(ch,n_mels=96,fmax=11025,n_fft=1024,hop_length=256,power=1)
                    mel = librosa.core.amplitude_to_db(mel)
                    predict.append(mel)
            predict = np.expand_dims(np.array(predict),-1)
            y_pred = prediction(predict)
        
            acc1 = average_accuracy(y_pred,y_label)
            acc2 = existance_accuracy(y_pred,y_label)
            
            sum_acc1 = sum_acc1 + acc1
            sum_acc2 = sum_acc2 + acc2

            print("Predicted track    : ",f)
            print("Expected output    : ", y_label)
            print("Predicted output   : ", y_pred)
            print("Percentage Accuracy: ", acc1,"%")
            print("Existance Accuracy : ", acc2,"%")
            print("")
            
        if (it == end): break
        it = it + 1
        text = not text
    return [sum_acc1,sum_acc2]

In [76]:
start = 1
end = 10
[sum_acc1,sum_acc2] = predict_songs(start,end)

aver_acc1 = round(sum_acc1/(end-start),2)
aver_acc2 = round(sum_acc2/(end-start),2)

print("------------------------------------------------------")
print("Average Percentage Accuracy among",end-start,"songs: ",aver_acc1,"%")
print("Average Existance  Accuracy among",end-start,"songs: ",aver_acc2,"%")

Predicted track    :  (02) dont kill the whale-11.wav
Expected output    :  [0, 0, 1, 0, 0, 0]
Predicted output   :  [0.22 0.11 0.66 0.8  0.06 0.44]
Percentage Accuracy:  67.17 %
Existance Accuracy :  66.67 %

Predicted track    :  (02) dont kill the whale-12.wav
Expected output    :  [0, 0, 1, 0, 0, 1]
Predicted output   :  [0.13 0.05 0.59 0.76 0.05 0.67]
Percentage Accuracy:  71.17 %
Existance Accuracy :  66.67 %

Predicted track    :  (02) dont kill the whale-13.wav
Expected output    :  [0, 0, 1, 0, 0, 1]
Predicted output   :  [0.19 0.11 0.57 0.83 0.11 0.55]
Percentage Accuracy:  64.67 %
Existance Accuracy :  66.67 %

Predicted track    :  (02) dont kill the whale-14.wav
Expected output    :  [0, 0, 1, 0, 0, 1]
Predicted output   :  [0.27 0.1  0.6  0.74 0.14 0.41]
Percentage Accuracy:  62.67 %
Existance Accuracy :  33.33 %

Predicted track    :  (02) dont kill the whale-15.wav
Expected output    :  [1, 0, 1, 0, 0, 0]
Predicted output   :  [0.32 0.05 0.97 0.47 0.12 0.33]
Percentage 